In [498]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from collections import Counter
from itertools import chain
from collections import defaultdict

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier


In [499]:
train = "train.csv"
test = "test.csv"
labels = "labels.csv"

train_set = pd.read_csv(train,encoding='unicode_escape')
test_set = pd.read_csv(test,encoding='unicode_escape')
label_set = pd.read_csv(labels,encoding='unicode_escape')


In [500]:
item_words = list()
train_set = train_set.replace(np.nan, '', regex=True)
item_words += list(chain(*train_set['line_item_name'].str.replace('[^\w\s]','').str.lower().str.split().tolist())) # line items
item_words += list(chain(*train_set['line_item_description'].str.replace('[^\w\s]','').str.lower().str.split().tolist())) # description
item_words += list(chain(*label_set['canonical_line_item_name'].str.replace('[^\w\s]','').str.lower().str.split().tolist()))  # labels

words_counter = Counter(item_words)

item_words = list(set(item_words))



Some observation of the data:
1. Many raw line items are at least partially the same as canonical line item
2. The hard cases are those lines where raw line items are substantially different from the canonical line item.

Basic idea:
1. The simple cases can be easily addressed with similarity comparisons, such as looking at cosine similarity
2. Harder cases may require additional information, such as vendor name. A vendor name only maps to a finite number of canonical line items.
3. Use cosine similarity to filter out the easy case first, then build a binary classifier for the rest (given raw line item and canonical line item pair, predict if they are from a same line item)

In [501]:
# create a vector for each line_item_name + description, as well as canonical_line_item_name

def line2vec(lineItem,vocab=item_words,description=None):
    # lineItem is a list of words in line_item
    nVocab = len(vocab)
    if description:
        lineItem = lineItem+description
    word2idx = {w:i for i,w in enumerate(vocab)}
    lineVec = np.zeros((1,nVocab+1))
    for w in lineItem:
        if w in word2idx:
            lineVec[0,word2idx[w]] = 1
        else:
            lineVec[0,-1] = 1
    return lineVec



In [502]:
# compute the vector representation of labels, construct as a matrix of dimension n_labs x k_dim
# then for each line item input, compute the cosine similarities and find the max

all_labs = label_set['canonical_line_item_name'].tolist()
lab2idx = {}
nlabs = len(all_labs)
lab_mat = np.zeros((nlabs,len(item_words)+1))
for j,l in enumerate(all_labs):
    lab2idx[l] = j 
    l = re.sub('[^\w\s]','',l).lower().split()
    lineVec = line2vec(l,vocab=item_words,description=None)
    lab_mat[j,:] = lineVec
#lab_mat = sparse.csr_matrix(lab_mat)


In [503]:
def cosinVal(lineVec,lab_mat):
    outVec = np.zeros(lab_mat.shape[0])
    for i in range(lab_mat.shape[0]):
        similarity = cosine_similarity(lineVec,lab_mat[None,i])
        outVec[i] = similarity
    return np.max(outVec), np.argsort(outVec)[::-1][:10]
    

In [504]:
# One good thing to do before moving forward is to build a dictionary between vendor and canonical line item
# because many vendors only have limited number of canonical line item variations

# feature space: words in line_item and vendor names
vendor2idx = {}
vendors = list(set(label_set['canonical_vendor_name']))
for j,v in enumerate(vendors):
    vendor2idx[v] = j

nlabs = len(label_set.index)
vendor2item = defaultdict(list)
for k in range(nlabs):
    vendor = label_set['canonical_vendor_name'][k]
    vendor2item[vendor].append(label_set['canonical_line_item_name'][k])
    
# then in training classifiers below, examples can be constructed only from the possible mappings
    


In [505]:
# now for the training set, for each row, compute the cosine similarity between all possible labels
# then decide take the most similar label, or pass down for the classifier with a pre-defined threshold
# construct positive and negative examples based on the true pair and possible pair based on similarity score

threshold = 0.4
for_classify = []
predicted = []
nrows = len(train_set.index)
for j in range(nrows):
    lineItem = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineDescrpt = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineVec = sparse.csr_matrix(line2vec(line,vocab=item_words,description=lineDescrpt))
    vendor = train_set['canonical_vendor_name'][j]
    possible_labs = vendor2item[vendor]
    labIds = [lab2idx[lab] for lab in possible_labs]
    subMat2matId = {i:lab2idx[lab] for i,lab in enumerate(possible_labs)}
    subMat = lab_mat[labIds,:]
    maxVal, maxIdx = cosinVal(lineVec,subMat)
    if maxVal<threshold:
        for_classify.append((j,[subMat2matId[k] for k in maxIdx[:5]])) # choose the first 10, see what happens
    else:
        predicted.append((j,subMat2matId[maxIdx[0]]))
        

In [506]:
# By just looking at cosine similarity can give us a decent performace
# Then I will use the miss-classified items from here and the remaining items to train a classifier

wrong = []
wrong_item = []
for a,b in predicted:
    if train_set['canonical_line_item_name'][a] != label_set['canonical_line_item_name'][b]:
        wrong.append(a)
        wrong_item.append(label_set['canonical_line_item_name'][b])
er = len(wrong)/len(predicted)
print("Error rate based on cosine only: %.3f" % er)

for i in wrong:
    lineItem = re.sub('[^\w\s]','',train_set['line_item_name'][i]).lower().split()
    lineDescrpt = re.sub('[^\w\s]','',train_set['line_item_name'][i]).lower().split()
    lineVec = sparse.csr_matrix(line2vec(line,vocab=item_words,description=lineDescrpt))
    vendor = train_set['canonical_vendor_name'][j]
    possible_labs = vendor2item[vendor]
    labIds = [lab2idx[lab] for lab in possible_labs]
    subMat2matId = {i:lab2idx[lab] for i,lab in enumerate(possible_labs)}
    subMat = lab_mat[labIds,:]
    maxVal, maxIdx = cosinVal(lineVec,subMat)
    for_classify.append((j,[subMat2matId[k] for k in maxIdx]))


Error rate based on cosine only: 0.058


In [507]:
# then collect the items in for_classify to train a binary classifier
import pdb

label2idx = {}
nlabs = len(label_set.index)
for j in range(nlabs):
    label2idx[label_set['canonical_line_item_name'][j]] = j
idx2label = {label2idx[lab]:lab for lab in label2idx}
    
X = []
Y = []

not_in_five = []

# build examples
for j,ids in for_classify:
    vendorVec = np.zeros(len(vendor2idx))
    #vendor = train_set['canonical_vendor_name'][j]
    #vendorVec[vendor2idx[vendor]] = 1
    #vendorVec = sparse.csr_matrix(vendorVec)
    trueLab = train_set['canonical_line_item_name'][j]
    lineItem = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineDescrpt = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineVec = line2vec(line,vocab=item_words,description=lineDescrpt)
    curr_y = 0
    for i in ids:
        labVec = lab_mat[i,:]
        feat = np.concatenate([lineVec.flatten(),labVec])
        if label_set['canonical_vendor_name'][i]==train_set['canonical_line_item_name'][j]:
            y = 1
            curr_y = 1
        else:
            y = 0
        X.append(feat)
        Y.append(y)
        # add the true lab if none of the similar indices is true label
    if curr_y==0:
        not_in_five.append(j)
        trueLabId = label2idx[train_set['canonical_line_item_name'][j]]
        trueLab = lab_mat[trueLabId,:]
        feat = np.concatenate([lineVec.flatten(),trueLab])
        X.append(feat)
        Y.append(1)
        
X = np.array(X)
Y = np.array(Y)

print(X.shape)
        

(1383, 5170)


In [508]:
# Maybe RF is preferable compared to SVM--too slow to train
# with different class weights

weights = {0:0.05, 1:1.0}
rf = RandomForestClassifier(n_estimators=1000, class_weight=weights,n_jobs=-1)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(rf, X, Y, scoring='f1', cv=cv, n_jobs=-1)
# summarize performance
print('Mean F1: %.3f' % np.mean(scores))

Mean F1: 0.463


In [510]:
# The F1 score doesn't look great. However, since the majority (~60% or more) of the instances are expected to be
# classified with high accuracy, then the rest shouldn't hurt that much. Also given the limited possibilities given
# the name of vendor, it shouldn't add too much burden for manually fixing the errors, or using some heuristics on
# top of the classification result.
# then fit the model with full dataset
rf.fit(X,Y)

# For the eval set, first do the same thing as the training set
# Then select label from the possible label set with the trained RF.

threshold = 0.4
for_rf = []
y_hat = []
nrows = len(test_set.index)
for j in range(nrows):
    lineItem = re.sub('[^\w\s]','',test_set['line_item_name'][j]).lower().split()
    lineDescrpt = re.sub('[^\w\s]','',test_set['line_item_name'][j]).lower().split()
    lineVec = sparse.csr_matrix(line2vec(line,vocab=item_words,description=lineDescrpt))
    vendor = test_set['canonical_vendor_name'][j]
    possible_labs = vendor2item[vendor]
    labIds = [lab2idx[lab] for lab in possible_labs]
    subMat2matId = {i:lab2idx[lab] for i,lab in enumerate(possible_labs)}
    subMat = lab_mat[labIds,:]
    maxVal, maxIdx = cosinVal(lineVec,subMat)
    if maxVal<threshold:
        for_rf.append((j,[subMat2matId[k] for k in maxIdx]))
    else:
        y_hat.append((j,subMat2matId[maxIdx[0]]))

#construct the test sets
X_test = []
labidlist = []
xtest2labid = {}
for j,ids in for_rf:
    vendorVec = np.zeros(len(vendor2idx))
    trueLab = train_set['canonical_line_item_name'][j]
    lineItem = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineDescrpt = re.sub('[^\w\s]','',train_set['line_item_name'][j]).lower().split()
    lineVec = line2vec(line,vocab=item_words,description=lineDescrpt)
    for i in ids:
        labidlist.append((j,i))
        labVec = lab_mat[i,:]
        feat = np.concatenate([lineVec.flatten(),labVec])
        X_test.append(feat)

for j,(r,idx) in enumerate(labidlist):
    xtest2labid[j] = (r,idx)
    
Y_pred = rf.predict(X_test)
for i,h in enumerate(Y_pred):
    if h==1:
        r,idx = xtest2labid[i]
        y_hat.append((r,idx))
        

In [511]:
# The last step: deal with two possible prediction errors:
# 1. one line item assigned to multiple canonical line items --> pick the one with highest cosine
# 2. one line item assigned to none of the canonical line items --> pick the one with highest cosine

predicted = list(sorted(y_hat))
predDict = defaultdict(list)

for j,h in predicted:
    predDict[j].append(h)
    
out_lab_list = []
    
for i in range(len(test_set.index)):
    if len(predDict[i]) == 0:
        vendor = test_set['canonical_vendor_name'][i]
        possible_labs = vendor2item[vendor]
        labIds = [lab2idx[lab] for lab in possible_labs]
        subMat2matId = {i:lab2idx[lab] for i,lab in enumerate(possible_labs)}
        subMat = lab_mat[labIds,:]
        maxVal, maxIdx = cosinVal(lineVec,subMat)
        out_lab_list.append(subMat2matId[maxIdx[0]])
        
    elif len(predDict[i])>1:
        cos = 0
        arg = predDict[i][0]
        lineItem = re.sub('[^\w\s]','',train_set['line_item_name'][i]).lower().split()
        lineDescrpt = re.sub('[^\w\s]','',train_set['line_item_name'][i]).lower().split()
        lineVec = line2vec(line,vocab=item_words,description=lineDescrpt)
        for idx in predDict[i]:
            labvec = lab_mat[idx]
            sim = cosine_similarity(lineVec,labvec.reshape(1,labvec.shape[0]))
            if sim>cos:
                cos = sim
                arg = idx
        out_lab_list.append(arg)
    else:
        out_lab_list.append(predDict[i][0])
            

In [512]:
# write to test_set

out_labs = [idx2label[i] for i in out_lab_list]
test_set['canonical_line_item_name'] = pd.Series(out_labs)
test_set.to_csv("results.csv")
